In [1]:
# !pip install gensim --user

In [715]:
import pickle

In [366]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [505]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
import re
import numpy as np
import pandas as pd
from string import punctuation

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [ ]:
from PreProcessing.find_common_part import find_common_part
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS, get_EBS_entered, get_STT1_Google_entered, get_STT1_Google, get_STT1_Transcribe_entered

In [4]:
from jamo import h2j, j2hcj, j2h
from soynlp.hangle import levenshtein, jamo_levenshtein
import nltk
from nltk import Text
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer, regexp_tokenize
from nltk.help import upenn_tagset
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)

0
0
0
1
2
5
4
8


In [5]:
EBS = get_EBS(1, 26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [6]:
GOOGLE = get_STT1_Google(1, 26)

['lec00001_STT1_Google.txt', 'lec00002_STT1_Google.txt', 'lec00003_STT1_Google.txt', 'lec00004_STT1_Google.txt', 'lec00005_STT1_Google.txt', 'lec00006_STT1_Google.txt', 'lec00007_STT1_Google.txt', 'lec00008_STT1_Google.txt', 'lec00009_STT1_Google.txt', 'lec00010_STT1_Google.txt', 'lec00011_STT1_Google.txt', 'lec00012_STT1_Google.txt', 'lec00013_STT1_Google.txt', 'lec00014_STT1_Google.txt', 'lec00015_STT1_Google.txt', 'lec00016_STT1_Google.txt', 'lec00017_STT1_Google.txt', 'lec00018_STT1_Google.txt', 'lec00019_STT1_Google.txt', 'lec00020_STT1_Google.txt', 'lec00021_STT1_Google.txt', 'lec00022_STT1_Google.txt', 'lec00023_STT1_Google.txt', 'lec00024_STT1_Google.txt', 'lec00025_STT1_Google.txt', 'lec00026_STT1_Google.txt']


In [593]:
EBS[1070:3000]
# EBS

'게 수선을 똑같이 그어 주면 바로 두 직각삼각형 ABC하고, ADE는 서로 어떤 닮음?AA닮음이 된다. 이건 우리가 중학교 2학년 때 배웠죠, 그렇죠? ∠A가 공통이고, 그다음에 둘 다 직각을 갖고 있으니까 AA닮음이 될 테고.우리가 중2 때 배웠던 닮음비라는 건 뭐였냐 하면 대응하는 변의 길이의 비였어요, 그렇죠? 그래서 만약에 이 길이를 5라고 해 보고, 이 길이를 10이라고 해 보면.두 삼각형의 닮음비는 5:10. 1:2겠죠. 그렇다면 우리가 이 길이를 4라고 놔 보면, 이 길이는 얼마가 됩니까, 당연히? 그렇죠. 4의 두 배인 8이 돼야 돼.이 길이를 거꾸로 우리가 6이라고 해 보겠습니다. 그럼 거꾸로 이 작은 쪽은 얼마가 될까? 1:2, 2:1. 얼마? 3이 돼야 된다는 걸 여러분들이 알 수 있죠.그래서 우리는 닮음비라는 건 ‘작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비’이니까 5:10, 3:6, 4:8. 이것이 모두 다 1:2로 같고 그것을 닮음비라고 한다.이렇게 공부를 했었는데 이것 말고도 이러한 것들도 성립할 거란 얘기예요. 예를 들어 작은 삼각형에서 빗변 분의 높이 얼마입니까? 5분의3이죠?큰 삼각형에서 빗변 분의 높이 얼마죠? 10분의6이죠. 이 둘 어때요? 5분의3으로 서로 같다는 거 확인할 수 있고.또 마찬가지로 작은 삼각형에서 예를 들어서 빗변 분의 밑변 해 봅시다. 5분의4죠. 큰 삼각형에서 빗변 분의 밑변 해 보면 얼마죠?10분의8이죠. 근데 이것도 서로 어때? 같아요. 마찬가지로 하나 더 해 볼까요? 작은 삼각형에서 밑변 분의 높이는 4분의3이죠.큰 삼각형에서 밑변 분의 높이는 어때요? 8분의6인데, 이들이 모두 다 어때? 같다는 걸 여러분들이 알 수 있단 얘기예요. 결국 지금 이 둘의 차이는, 같은 거 같아 보이나요? 조금 달라요.작은 삼각형과 큰 삼각형의 두 대응변의 길이의 비를 이야기하는 게 닮음비고, 한 삼각형에서 어떤 두 변을 뽑아서 그 둘의 비랑 큰 삼각형에서 그 둘의 길이의 비는비가 서로 같은 성질이 있다는 게

In [609]:
GOOGLE[1000:3000]
# GOOGLE

' 수선을 똑같이 보여 주면 바로 2차각성 가평 a b a d는 서로 어떤 닮은 a 닮음이 된다 이건 우리가 중학교 2학년 때 가게가 통이고 그다음에 둘 다 치과를 갖고 있으니까 a 삶이 될 테고 우리가 주의 때문에 어떤 닮음비 일하는 건 뭐냐면 대응하는 변의 길이의 비 왔어요 그래서 그래서 만약에 이게들을 오라고 해 보고 얘기를 11분 삼각형의 닮음 비는 5대 11대 이겠죠 자 그렇다면 우리가이 길을 사라고 나 보면 이게 된 얼마나 됩니까 당연히 그쵸 5일 8일 해야 돼 진짜이 길을 걷고 우리가 6이라고 해보겠습니다 그럼 거꾸로 작은 쪽은 얼마나 됐다 1대 2대 1 얼마 3일 돼야 된다는 걸 여러분들이 할 수 있죠\n 우리는 닮은 비라는 건 작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비니까 오대식 3대 6\n 4대 8 이것이 모두 다 일대일로 갖고 그것을 닮음비라고 한다 이렇게 공부했었는데 이것 말고도 이런 것들도 성립할 거라 예를 들어 작은 삼각형에서 빗변분의 높이 얼마입니까 큰 삼각형에서 빗변분의 높이 얼마죠 숯불에 유기적 이들 어때요 5분의 3으로 서로 같다라는 확인할 수 있고 또 마찬가지로 자는 삼각형에서 예를 들어서 빗변분의 밑변에 봅시다 5분의 4 적 큰 삼각형에서 변 분에 밑변에 보면 얼마죠 10분에 반이죠 근데 이것도 사러 때 같아요 마찬가지로 하나 더 해 볼까요 작은 삼각형에서 밑변 불에 높이는 4분의 3 이적 큰 삼각형에서 미평 그래도 필요 없대요 8분의 6인데 이들이 모두 다 어때 같다란 걸려 놈들이 알 수 있단 얘기야 결국 지금이 둘의 차이는 같은 거 같아 보이나요\n 달라요 작은 삼각형과 큰 삼각형의 두 대는 변의길이 비를 이야기 한게 닮음비고 1 삼각형에서 어떤 두 병을 뽑아서 그들의 비람 큰 삼각형 둘레 길이 8m 서로 비가 서로 같은 성질이 있다라는게 바로 닮은 직각삼각형에서 성질입니다 이제 우리 가요 내용은 당연히 받아들일 수 있을 테니이 내용에 대해서 간단하게 확인 문자 한번 살펴보고 가도록 하겠습니다 오른쪽 그림과 같은

In [9]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

def insert_dot(text, dot):
    pattern1 = re.compile(r'\S(ㄷㅏ)$')   #다
#     pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
#     pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
#     pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
#         elif pattern2.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _2 += 1
#             list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
#         elif pattern4.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _4 += 1
#         elif pattern5.findall(new_):
#             text_list.append(_.replace(_,_+dot))
#             _5 += 1
        else:
            text_list.append(_)
            _6 += 1
#     print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

# EBS 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
#     ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
#     ebs_whitespace_removed = re.sub('[\s]+', '', ebs_punc_removed)

    return ebs_punc_removed

# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
#     google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
#     google_whitespace_removed = re.sub('[\s]+', '', google_punc_removed)

    return google_punc_removed

In [651]:
ko_pre_EBS = EBS_preprocessing(EBS) #한글로 치환
ko_pre_GOOGLE = GOOGLE_preprocessing(GOOGLE) #한글로 치환
dot_inserted_EBS = ' '.join(insert_dot(ko_pre_EBS,'.')).split('.')
dot_inserted_GOOGLE = ' '.join(insert_dot(ko_pre_GOOGLE,'.')).split('.')

In [652]:
print(len(dot_inserted_EBS))
print(len(dot_inserted_GOOGLE))

2939
2752


In [658]:
ws_removed_EBS = [''.join(_.split()) for _ in dot_inserted_EBS]
ws_removed_GOOGLE = [''.join(_.split()) for _ in dot_inserted_GOOGLE]

In [712]:
most_similar_GOOGLE = []
for eidx, ebs_stnc in enumerate(ws_removed_EBS):
    MIN = 150
    for google_stnc in ws_removed_GOOGLE[eidx-200 if eidx-200 > 0 else 0:eidx+200 if eidx+200 < len(ws_removed_GOOGLE)-1 else len(ws_removed_GOOGLE)-1]:
        current = levenshtein(ebs_stnc, google_stnc)
        if current < MIN:
            MIN = current
            MIN_stnc = google_stnc
    most_similar_GOOGLE.append(MIN_stnc)
#     if MIN/(len(ebs_stnc)+len(google_stnc)) > 0.7:
#         print('!!!!!!!!!!!!!!!!!!!!!!??!????????????????')
    print(MIN, '         ', MIN/(len(ebs_stnc)+len(google_stnc)))
    print()
    print('ebs : ',ebs_stnc)
    print()
    print('google : ',MIN_stnc)
    print()
    print('*******'*10)

0           0.0

ebs :  반갑습니다

google :  반갑습니다

**********************************************************************
7           0.08974358974358974

ebs :  이비에스친구여러분들여러분들과함께중학뉴런수학삼하를강의하게된손석민입니다

google :  이비에스여러분들여러분들과함께중학뉴런수학삼할을강의하기됨손성민입니다

**********************************************************************
2           0.013888888888888888

ebs :  이학년뉴런을통해서도만났었는데요여러분들이렇게다시만나게되어서정말반갑습니다

google :  이학년이런을통해서도만났었는데여러분들이렇게다시만나게되어서정말반갑습니다

**********************************************************************
6           0.043795620437956206

ebs :  앞으로삼하과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거고요오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하과정이특히어떻게수업되는지확인할수있을겁니다

google :  앞으로삼과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거구요오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하마종이접기어떻게수업되는지확인할수있을겁니다

**********************************************************************
4           0.0547945205479452

ebs :  일단삼학년과정같은경우에는크게삼개의대단원으로이루어져있는데요첫번째가삼각비라는단원이고두번째가원의성질세번째가통계이렇게이루어져있습니다

google :  일단삼학년가정같은경우에는크게세개의대단원으로이루어져있는데요첫번째가삼각비라는단어

30           0.2727272727272727

ebs :  그각각을사인코사인탄젠트라고이름지었고그기준을잡을건데첫번째그내용에앞서서그러면우리는이변을높이라고생각하고이변을지금밑변이라고생각하고이변을빗변이라고생각합니다

google :  꼭그기준을잡을건데어첫번째그내용이없어서그러면우리는이변을높이라고생각하고이별을지금이변이라고생각이별을편이라고생각합니다

**********************************************************************
5           0.09259259259259259

ebs :  근데기준각이라는게앞으로등장을해요일단기준각이라는게나오나보다

google :  근데기준각일하는게앞으로등장을해요아일단기준각일하는게나오나보다

**********************************************************************
7           0.03977272727272727

ebs :  어떤각을기준으로해서이야기하는구나그런데기준각에서마주보는변을우리는높이라고약속했습니다

google :  어떤값을기준으로해서이야기하는구나그런데기준가게에서마주봄우리는높이라고약속했습니다

**********************************************************************
35           0.2536231884057971

ebs :  예를들어서각에이를기준각이라고놔보면그럼일단씨는직각에서마주보는변이빗변이라는건우리가알고있어요그렇죠한글로용어를좀정리해볼텐데이기준각에서마주보는변을우리는뭐라고했냐면높이라고이야기를한거고그럼나머지한변은자동으로뭐가되죠밑변이됩니다

google :  예를들어서가게를지금가기라고나보면그럼일단수있는집가게에서마주보이는변이빗변일하는건알고있어요그죠근데한글로영어를좀정리해볼텐데이기준가겠어맛있어보이는별을우리는뭐라고했냐면높이라고이야기를한거고그럼나머지한번은자동으로뭐가늦어이변이됩니다

******************************************

29           0.17682926829268292

ebs :  우리함께가보도록할게요각비가구십도인직각삼각형이있는데길이가육팔십이라는게표현이되어있고요코사인에이의값을구하세요했습니다

google :  우리함께가보도록할게요작비구십도어여기가구십도인직각삼각형의있는데육백팔십있다가지금표현해있고요코사인수고하셨습니다

**********************************************************************
150           0.19329896907216496

ebs :  바로답나오죠우리코사인은뭐분의뭐였습니까아직암기안하셨죠빗변분의밑변이었어요코사인에이의값이라는건삼각비세변중에두변을뽑는방법중에빗변분의밑변의길이의비를코사인이라고하더라근데밑변이라는건뭐냐그냥밑에가로로있으면밑변이아니라기준각에대해서대변이높이였어요그러면여기서지금에이가기준각이니까기준각에대해서마주보는변이높이그리고직각에서마주보는게빗변임은확실하잖아요그렇죠그럼나머지이변이바로뭐가되는거야밑변이되는거예요그럼빗변분의밑변이니까십분의육이네십분의육은약분해서뭐라고써주면됩니까오분의삼선택지에서정답은일번이구나이렇게구하면되는거고만약에탄젠트값은얼마예요탄젠트에이라고하면밑변분의높이이니까삼분의사라고답을하면되겠죠사인에이는구하라고한다면만약에십분의팔이니까빗변분의높이죠오분의사이렇게여러분들께서답을구할수있으면되는거예요어렵지않죠그렇죠두번째예제이번으로한번들어가볼텐데이제여기서부터는조금여러분들이신경을써야되죠왜냐하면세변의길이가주어져있고사인코사인탄젠트값을구하세요다시한번사인코사인탄젠트처음들어봐서익숙하진않지만어쨌든세변중에사인코사인탄젠트값을구하세요라는말은그직각삼각형에서빗변분의높이의비를구하세요빗변분의밑변의비를구하세요밑변분의높이의비를구하세요이말을간단하게쓴게사인코사인탄젠트값을구하라는얘기예요됐나요그럼같이함께예제이번보겠습니다

google :  우리함께가보도록할게요작비구십도어여기가구십도인직각삼각형의있는데육백팔십있다가지금표현해있고요코사인수고하셨습니다

*******************************************************

31           0.6458333333333334

ebs :  우리가오늘처음개념으로배웠던게어디에쓰이는지여러분들께서확인할수있는그런유형이었습니다

google :  이렇게여러분들께서답을구하시는유형입니다

**********************************************************************
9           0.08490566037735849

ebs :  이제이렇게예제까지살펴봤는데조금더다양한유제들한번살펴봐야되겠죠유제풀이시작합니다

google :  제가이제이렇게예데까지살펴봤는데좀더다양한주제를한번봐야되겠죠유제풀이시작합니다

**********************************************************************
16           0.1553398058252427

ebs :  여러분들께미리하나부탁드리는데이예제와유제풀이이사이있죠인터넷으로듣고있다면반드시여러분들이유제를직접풀고그리고강의를들어주시기바랍니다

google :  테런느낌이려나부탁드리는데이유재예제와유제품은이사있죠인터넷으로듣고있다면반드시여러분들이직접풀고그리고강의를들어주시기바랍니다

**********************************************************************
150           0.18679950186799502

ebs :  선생님이앞으로항상내주는그런숙제라고생각하시면돼요유제일번부터보도록할텐데직각삼각형에이비씨에서코사인씨를구하래요결론부터보고우리가거꾸로가볼게요코사인은뭐분의뭐빗변분의밑변그럼여기서직각에서마주보는변이빗변이고밑변은마주보는게높이겠죠기준각에서그러니까자동으로이변이밑변이야그러면코사인씨라는건빗변분의밑변이니비씨분의에이씨를하라는얘기고비씨는몰라그리고에이씨는얼마야삼루트오야그러니까이문제는결국누굴구하라는얘기야비씨를구하라는얘기예요코사인씨구하라는게결국비씨구해서해결해라이얘기죠그럼비씨를구할수있는방법은뭡니까직각삼각형이니까당연히피타고라스의정리이죠그럼비씨는어떻게된다

google :  테런느낌

40           0.1038961038961039

ebs :  여기까지갈수있다면이제사인에이라는건뭐예요사인에이빗변분의높이잖아그렇죠결국이렇게내려와보면칠분의이루트육이죠곱하기탄젠트씨여기서중요한건뭐야기준각이바뀌었어씨로이게기준각이래그러면마주보는게높이죠그리고이변이밑변이죠그러니까탄젠트씨는밑변분의높이하려면이루트육분의오가되는겁니다

google :  여기까지갈수있다면이제쌓이네일하는건뭐야싸인에이빗변분의피자나그죠아결국이렇게내려가보면칠분의이유이죠탕정택시여기서중요한거야기준가게바뀌었어씨로이게그건가그래그러면마주보이는게높이죠그리고이변이몇번이죠그러니까탄젠트씨는밑변브레노피하려면이루트육분의오가되는겁니다

**********************************************************************
16           0.06201550387596899

ebs :  이렇게이거주의해야죠기준각이에이에서씨로바뀌고있습니다

google :  이렇게에이사백칠십오바뀌고있습니다

**********************************************************************
0           0.0

ebs :  결국정답은뭐가돼칠분의오한번쯤다시들었으면하는문제입니다

google :  결국정답은뭐가돼칠분의오한번쯤다시들었으면하는문제입니다

**********************************************************************
7           0.056

ebs :  유제오육번중에특히육번다시한번봤으면좋겠네요이제유제칠번으로들어가보도록하겠습니다

google :  뮤제오육번중에스케이육번다시한번봤으면좋겠네요유재칠번으로들어가보도록하겠습니다

**********************************************************************
150           0.23547880690737832

ebs :  우리직각에서빗변에수선을그은직각삼각형이있으면삼개의삼각형은닮음이

...

0           0.0

ebs :  오늘은삼각형의넓이그리고다각형의넓이이렇게함께알아보는시간입니다

google :  오늘은삼각형의넓이그리고다각형의넓이이렇게함께알아보는시간입니다

**********************************************************************
1           0.04

ebs :  첫번째삼각형의넓이에서부터시작합니다

google :  첫번째삼각형의넓이서부터시작합니다

**********************************************************************
1           0.030303030303030304

ebs :  우리가알고있는삼각형의넓이구하는공식한번얘기해봅시다

google :  자우리가알고있는삼각형의넓이구하는공식한번얘기해봅시다

**********************************************************************
16           0.24242424242424243

ebs :  삼각형넓이어떻게구해요그렇죠이분의일곱하기밑변곱하기높이다알고있어요그렇죠그러면우리는삼각형하나를딱그려놓고보겠습니다

google :  생각을어떻게돼요그래서이분의일곱하기사곱하기높이다알고있어요그래서아그러면우리는하나를딱그려놓고보겠습니다

**********************************************************************
15           0.11904761904761904

ebs :  삼각형하나를그려놓고이삼각형의넓이를구하고싶은데지금뭐가주어지는거냐면이밑변의길이하고잘보세요높이가주어지면딱좋겠죠이렇게두개가그런데이렇게두개가주어지는게아니라이거안알려준다는얘기예요이거안알려주고대신에뭘알려주냐이변의길이를알려줍니다

google :  생각하나그려놓고이삼각형의넓이를보고싶은데지금뭐가주어지는거냐면이미변의길이하고잘보세요높이가주어지면딱좋겠죠이렇게두개가근데이렇게두개가좋아지는게아니라이거안알려준다는얘기야이거안알려

6           0.2

ebs :  함께같이문제를살펴볼텐데이번문제한번보겠습니다

google :  함께같이문제를텐데이범재한번보겠습니다

**********************************************************************
11           0.28205128205128205

ebs :  사각형에이비씨디의넓이했어요다른문제들도여러분들반복하셔야합니다

google :  사각형에이비씨널고있어요다른분들도여름도반복하셔야됩니다

**********************************************************************
33           0.308411214953271

ebs :  이건어떻게풀까이렇게자르면안되겠죠왜각도를모르잖아잘린각도를그러니까우리는에이하고씨를이렇게연결해서결국전체사각형에이비씨디의넓이를삼각형에이비씨의넓이와삼각형에이씨디넓이의합으로바라보도록하겠습니다

google :  왜각도를모르잖아딸린각도를그러니까우리애기하고씨를이렇게연결해서결국전체사각형의넓이를삼각형에이비씨넓이와삼각형넓이의합으로알아보도록하겠습니다

**********************************************************************
53           0.7361111111111112

ebs :  삼각형에이비씨를먼저볼까어삼각형에이비씨를봤더니두변이이와육이고끼인각이주어져있네그럼우리는삼각형의넓이구할수있죠공식어떻게됐었다

google :  그냥두번째사각형의넓이를구하는거공부했는데사각형의넓이를우리는어떻게보냈다

**********************************************************************
38           0.6909090909090909

ebs :  이분의일곱하기두변의길이의곱곱하기사인끼인각그런데그끼인각이몇도백이십도알아몰라몰라어떻게했었다

google :  그냥두번째사각형의넓이를구하는거공부했는데사각형의넓이를우리는어떻게보냈다

*

19           0.6551724137931034

ebs :  오늘배웠던여러가지공식들정리하고마무리합니다

google :  이제한십오분쯤에스카이공식외워쓸거라고생각합니다

**********************************************************************
37           0.5138888888888888

ebs :  오늘첫번째개념이었죠삼각형의넓이삼각형넓이는당연히두변의길이의곱곱하기사인끼인각했는데앞에이분의일들어갔던거여러분들기억하실겁니다

google :  오늘배웠던여러가지음식들정리하고마무리여기서첫번째기념일삼각형의넓이삼각형의넓이는당연히두변의길이곱하기싸인각했는데앞에이분의들어왔던그런데기억하실겁니다

**********************************************************************
59           0.5566037735849056

ebs :  그다음두번째한번살펴보도록할텐데요둔각인경우에는어떻게했어요사인백이십도사인백오십도우리가이런값들을모르기때문에바깥쪽외각을이용했다는것기억하시면되겠죠그내용만기억하시면따로공식외우지않으셔도됩니다

google :  우리가이런것들을모르기때문에바깥쪽외곽을이용해서따라하시면되겠죠그내용만따로주지않으셔도됩니다

**********************************************************************
23           0.39655172413793105

ebs :  그다음다각형에서우리특히나뭘배웠습니까평행사변형공식이분의일빠지는이유대각선그었을때삼각형넓이의합이다

google :  다각형에서우리특히나뭘배웠으니까평행사변형공식빠지는이유대각선그때삼각형넓이합의도합격하시면되겠고요번쩍인다

**********************************************************************
25           0.5555555555555556

ebs :  기억하시면되겠고요일반적인다각형은대각선

52           0.31901840490797545

ebs :  이거이용할거예요됐나요그런데우리가곰곰이보면에이에이치의길이를구하라고했으니까이길이를높이니까에이치라고놔볼게요물론엑스라고놔도상관없어요그럼우리는어떻게해이길이도에이치로표현할거고이길이도에이치로표현할거예요그단계죠선생님이그때강의할때에도이것도몇에이치로표현하고이것도몇에이치로표현해서여기서에이치를구하겠다

google :  이거이용할거야됐나요그런데우리가공범이보면에이에이치길이를구하라고했으니까용인일을높이니까에이치하고나올게블랙쓰라고나도물은상관없어요그럼우린어떻게요길이도에이치로편할거고유빈이도에이치로편할거야그냥믿어그때강의할때도요것도몇에이치로펴라고이것도몇위치로여기서에이칠구하겠다

**********************************************************************
2           0.1111111111111111

ebs :  이렇게얘기를했었습니다

google :  이렇게얘기를썼습니다

**********************************************************************
108           0.5775401069518716

ebs :  그런데왼쪽은좀쉬운게직각이등변삼각형이죠구십도사십오도사십오도일테니까그럼이길이가자동으로뭐가돼에이치가돼요일에이치구나일생략된거예요비에이치그렇죠그다음씨에이치는우리가어떻게갈거냐아이각을가지고서하면어렵다고했죠계산이복잡하니까이각으로가자고했었고이각은몇도육십도였어요그러면탄젠트육십도가에이치분의씨에이치이렇게갔었죠탄젠트육십도가에이치분의씨에이치입니다

google :  근데왼쪽은좀추운게직각이등변삼각형이가구십사십오도사십오도일테니까그럼네가자동으로뭐가돼에이치가든응일에이치구나일생략된거예요그다음일층을우리가어떻게갈거냐어렵다

**********************************************************************
147           0.392

ebs :  기준각이여기있으니까밑변분의높이그렇죠그럼

90           0.33088235294117646

ebs :  평행사변형의일부분이죠그러니까우리는평행사변형의넓이를구하는공식을알고있다면특히네변의길이가같은평행사변형이잖아그렇죠그러니까이각이백삼십오도이면이각은몇도예요사십오도인거이건알수있죠더해서백팔십한변의길이를엑스라고놔보면이길이도엑스니까우리는평행사변형으로바라보면이거넓이어떻게구했어두변의길이의곱곱하기사인끼인각이었잖아그렇죠즉엑스곱하기엑스곱하기사인사십오도이게넓이인데그넓이가팔루트이라는이야기야그럼이게뭐죠이분의루트이엑스의제곱는팔루트이라는이야기니까이렇게나눠보고이배해주면엑스의제곱는십육이돼서엑스의값은얼마사가된다

google :  평행사변형의일부분에서그러니까우리는평행사변형의넓이구하는공식은알고있다면특히네변의길이가같은평행사변형거잖아그죠이더하기백삼십오되면네가돈몇도예요사십오도인거일어날수있죠저희백팔십한변의길이를엑스라고나보면이기도에쓰니까우리는평행사변형으로바라보면이거넓이어떻게구해써두변의길이의구곱하기사인게임가격저는그저엑스곱하기엑스곱하기싸인사십오도이게넓은데그넓이가팔루트이다라는이야기야이게뭐죠이분의루트이엑스제곱이팔루트이다

**********************************************************************
10           0.5555555555555556

ebs :  한변의길이는사씨미터다

google :  수고하셨습니다

**********************************************************************
18           0.6428571428571429

ebs :  이렇게여러분들이답을구할수있다는얘기입니다

google :  수고하셨습니다

**********************************************************************
30           0.6382978723404256

ebs :  조금부지런히왔는데왜그러냐면마지막도전고난도한번선생님이설명을해보도록하겠습니다

google :  다음문제육번문제한번

In [720]:
with open('ws_removed_EBS.txt', 'wb') as f:
    pickle.dump(ws_removed_EBS, f)

In [721]:
with open('most_similar_GOOGLE.txt', 'wb') as f:
    pickle.dump(most_similar_GOOGLE, f)

In [ ]:
# 불러올때
import pickle
with open('ws_removed_EBS.txt', 'rb') as f:
    data = pickle.load(f)
    
with open('most_similar_GOOGLE.txt', 'rb') as f:
    data = pickle.load(f)

In [638]:
for i, _ in enumerate(ws_removed_EBS[:200]):
    for j, google in enumerate(ws_removed_GOOGLE[i:i+3]):
        print(['EBS'])
        print(_)
        print()
        print(['GOOGLE'])
        print(google, j)
        print(levenshtein(_, google))
        print('*****************************************************************************')
        print()
        

['EBS']
반갑습니 이비에스친구여러분들여러분들과함께중학뉴런수학삼하를강의하게된손석민입니 이학년뉴런을통해서도만났었는데요여러분들이렇게 시만나게되어서정말반갑습니 앞으로삼하과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거고요오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하과정이특히어떻게수업되는지확인할수있을겁니 일단삼학년과정같은경우에는크게삼개의대단원으로이루어져있는데요첫번째가삼각비라는단원이고두번째가원의성질세번째가통계이렇게이루어져있습니

['GOOGLE']
반갑습니 이비에스여러분들여러분들과함께중학뉴런수학삼할을강의하기됨손성민입니 이학년이런을통해서도만났었는데여러분들이렇게 시만나게되어서정말반갑습니 앞으로삼과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거구요오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하마종이접기어떻게수업되는지확인할수있을겁니 일단삼학년가정같은경우에는크게세개의대단원으로이루어져있는데요첫번째가삼각비라는단어이거두번째가원의성질세번째가통계이렇게이루어져있습니 0
19
*****************************************************************************

['EBS']
반갑습니 이비에스친구여러분들여러분들과함께중학뉴런수학삼하를강의하게된손석민입니 이학년뉴런을통해서도만났었는데요여러분들이렇게 시만나게되어서정말반갑습니 앞으로삼하과정어떻게진행되는지에대해서는여러분들께서오리엔테이션강의를통해서확인할수있을거고요오리엔테이션강의는반드시듣고오셔야여러분들이삼학년과정이하과정이특히어떻게수업되는지확인할수있을겁니 일단삼학년과정같은경우에는크게삼개의대단원으로이루어져있는데요첫번째가삼각비라는단원이고두번째가원의성질세번째가통계이렇게이루어져있습니

['GOOGLE']
 우리가일학년하도마찬가지고이학년하도그래왔듯주로도형에대한이야기가삼화많이 루어지는구나라는걸생각하면될테구요오늘은그첫시간이니까삼각비삼각알아요알아요근데이제합쳐져있는삼각비라는새로운영화이개념은공부하기앞서서당연히삼각비의뜻부터시작하는것이첫시간입니 1
221
****

335
*****************************************************************************

['EBS']
 는게바로닮은직각삼각형에서갖는성질입니

['GOOGLE']
 라는게바로닮은직각삼각형에서성질입니 0
3
*****************************************************************************

['EBS']
 는게바로닮은직각삼각형에서갖는성질입니

['GOOGLE']
 이제우리가요내용은당연히받아들일수있을테니이내용에대해서간단하게확인문자한번살펴보고가도록하겠습니 1
45
*****************************************************************************

['EBS']
 는게바로닮은직각삼각형에서갖는성질입니

['GOOGLE']
 오른쪽그림과같은삼각형에이비씨안에있는작은단답형큰삼각형에이디가있습니 2
29
*****************************************************************************

['EBS']
 그렇 면이제우리가이내용은당연히받아들일수있을테니이내용에대해서간단하게확인문제한번살펴보고가도록하겠습니

['GOOGLE']
 이제우리가요내용은당연히받아들일수있을테니이내용에대해서간단하게확인문자한번살펴보고가도록하겠습니 0
6
*****************************************************************************

['EBS']
 그렇 면이제우리가이내용은당연히받아들일수있을테니이내용에대해서간단하게확인문제한번살펴보고가도록하겠습니

['GOOGLE']
 오른쪽그림과같은삼각형에이비씨안에있는작은단답형큰삼각형에이디가있습니 1
47
*****************************************************************************

['EBS']
 그렇 면이제우리가이내용은당연히받아들일수

680
*****************************************************************************

['EBS']
 는얘기야앞으로그림이돌아가있는걸한번생각해볼텐데이렇게그림이돌아가있 고생각해보겠습니 직각삼각형이고만약에우리가이각을기준각이라고놔보겠습니 그럼여러분들이이연습이먼저돼야돼요빗변은누구죠직각에서마주보는변이빗변이야근데중요한거가로로있으니까밑변이고새로가높이가아니라기준각이주어지면기준각에서마주보는변이뭐가된 높이가된 그리고나머지한변이뭐가된 밑변이된 이약속을여러분들이첫번째이삼각비라는걸공부하기에앞서서이약속을여러분들이익히셔야됩니 직각에서마주보는대변이빗변이고기준각이주어지면거기서마주보는변이높이고그럼나머지한변이저절로밑변이 라고먼저약속으로받아들이셔야돼요그렇 면이제우리는이상태에서한번시작을해볼텐데육개의비중에삼개를뽑았 고했어요나머지삼개는고등학교에가서배우는거예요첫번째는빗변분의높이가있겠죠빗변분의높이우리는이비를뭐라고하냐하면사인이라고부릅니 그리고사인이렇게쓰고기준각을이뒤에이렇게붙여주기로기호로약속을한거야그러면사인기준각이라고주어지면아그각에대해서빗변분의높이의비를사인이라고하는구나그러니까한글로만날빗변분의높이의비가얼마니이렇게묻는게아니라앞으로는사인에이가얼마니이렇게물어보면그것이의미하는게빗변분의높이의비를묻는거구나라고생각하시면되는거예요그 음빗변분의밑변을우리는뭐라고부르냐하면코사인이라고부릅니 그리고똑같이기준각을넣어주면돼요결국이직각삼각형에서빗변분의밑변의길이가얼마니라는이긴한글을줄여서우리는코사인에이가얼마니이렇게묻는거예요됐나요세번째로밑변분의높이의길이의비를우리는뭐라고부르냐하면탄젠트라고부릅니 한글로는됐죠일단사인코사인탄젠트그렇죠탄젠트는이삼개중에어떤둘을뽑아서밑변분의높이를우리가계산한것을탄젠트에이라고이야기를한

['GOOGLE']
 자여기서여기서한가지약속선생님이거밑에있으니까밑변이건새로서있으니까높이아닌가요당연히그렇지않 1
722
*****************************************************************************

['EBS']
 는얘기야

684
*****************************************************************************

['EBS']
 면만약에십분의팔이니까빗변분의높이죠오분의사이렇게여러분들께서답을구할수있으면되는거예요어렵지않죠그렇죠두번째예제이번으로한번들어가볼텐데이제여기서부터는조금여러분들이신경을써야되죠왜냐하면세변의길이가주어져있고사인코사인탄젠트값을구하세요

['GOOGLE']
  시한번정리하지만삼각비라는것은세명중에직각삼각형에서삼번중에두병을와서만들수있는길을상담이라고하고그총여섯가지중에세가지를우리가중학교때됐는데쌓이는빗변분의높이코사인은빛변탄젠트는밑변분의높이달하는이기본약속을여러분들이웨어주신상태에서시작을해야되는거예요그리고빗변을집가게에서마주보이는대변이고높이뭐라고요기준가게대변이높이 0
146
*****************************************************************************

['EBS']
 면만약에십분의팔이니까빗변분의높이죠오분의사이렇게여러분들께서답을구할수있으면되는거예요어렵지않죠그렇죠두번째예제이번으로한번들어가볼텐데이제여기서부터는조금여러분들이신경을써야되죠왜냐하면세변의길이가주어져있고사인코사인탄젠트값을구하세요

['GOOGLE']
 그리고나머지한개는먹어야된 밑변이된 여기까지여러분들이먼저좀새로운약속이니까지금이약속들을안길을함께가셔야됩니 나같이할게요내용에대해서한번예제를좀살펴보도록할텐데이렇게돼있어요저그림과같이각도가구십도인직각삼각형에이비씨에서네모안에알맞은것을써넣어라해놓고싸긴에이에스입니 차쌓이는모였습니까빗변분의높이약속이오셨 라고가져갈게요오늘수업끝날때쯤열려있을겁니 쌓이는질병들에높이높이기준가게에서마주보이는게높여써아그러면사일에일하고있으니까애기가계신가그래그러면마주보이는게우리가뭐라고요높이라고이건집가서마저보이는데편이죠나머지한별이자동으로먹어야된 그랬어밑변이된 라고있잖아요그죠그러면쌓이는몸매보 빗변분의높이이 십분에근데일단집으로오면에이씨분에피곤한걸알수있고보내버리니까약불에서오분의사오분의사가가게에대한싸인갑시 1
332
********

391
*****************************************************************************

['EBS']
  음으로예제삼번문제들어가보도록하겠습니 예제삼번을여러분들이왜어려워할까요일단문제읽어볼게요각비는구십도인직각삼각형에이비씨에서사인에이는삼분의이일때탄젠트씨의값이렇게돼있잖아요그냥쭉읽었어어떻게해야되지라는걱정이되죠왜그런생각이듭니까그림이없어요그래서뭘어떻게해야될지모르겠어요그럼우리는그림을직접그리는거예요문제에서뭐라고했어각비는구십도인직각삼각형이렇게돼있잖아요그렇죠각비는구십도인직각삼각형뭐이렇게한번그려보는거예요각비라고놓으면돼그렇죠그럼여기를에이라고놓고여기를씨라고놔보는거예요근데사인에이는삼분의이라는조건이주어져있네기준각이거야에이에대해서그러면사인은뭐분의뭐였습니까빗변분의높이였어요그럼기준각에서마주보는게높이네그럼사인에이라는건빗변분의높이네근데그건뭐래요삼분의이래요그러면이게삼일때이게이라는얘기구나여기까지갈수있겠죠그럼우리누굴구할수있죠당연히에이비를누굴통해서피타고라스의정리를통해서에이비의길이를구할수있고요에이비는루트빗변의제곱빼기

['GOOGLE']
 문제에서뭐라그랬냐직각삼각형에이비씨에서지금높이선생님높이라고했지만사실앞으로이제우리가관점을달리해서이게높이인지아닌지아직모르는거예요그죠우리가지금까지높이라고했고밑변이라고했지만비가기준각일하면높이가되는겁니 2
393
*****************************************************************************

['EBS']
 른변의제곱이니까루트오가된 는걸여러분들이구해놓고시작하는겁니 그런데구하라는건뭐예요탄젠트씨예요기준각이누구로바뀌었어씨로바뀌었어이건무슨얘기야그럼우리는 시한번정리를해야되겠죠어차피직각에서마주보는변이빗변이고기준각에서마주보는변이높이였죠그리고이변이밑변이되는겁니 기준각이바뀌면밑변높이가뒤바뀌는거예요근데탄젠트라는건뭐분의뭐였어밑변분의높이였잖아그러니까뭐가된 는얘기야밑변분의높이는이분의루트오이렇게되겠죠탄젠트씨라는것은바로누구분의누구 비씨분의에이비이니까밑변분의높이니까이분의루트오를여러분들이선택

...

267
*****************************************************************************

['EBS']
 리꼴이죠그러니까또에이비하고디씨까지등변사 리꼴의성질같 는거까지주어져있습니 그리고길이가삼오칠등이주어져있고요사인비더하기탄젠트비의값그러니까여러분이이런문제를보면이거읽 가이미뭐야되게복잡해보여그런데사인하고탄젠트비의값을어떻게구해한가지원칙만기억하시면돼요뭐예요삼각비구하래요뭐만들어야해직각삼각형이게끝인거야됐나요직각삼각형을만들어야해요여기서이사 리꼴에서그런데각비에대해서니까각비를포함하는직각삼각형을만든 이게그냥핵심이에요그러면각비를포함하는직각삼각형은당연히이렇게만들어야지이거잖아그렇죠그러면여기에서도똑같이그렇죠이거여러분이풀이는워낙유명해왜등변사

['GOOGLE']
 는생각비가없기때문에타블라스정리를이용해야되는구나피타고라스정리해서에이씨길이는바로갈수있나요이제곱마이너스 른데를제곱이겠죠루트이의제곱마이너스루트육의제곱마이너스이의제곱이니까루트육마이너스사가돼서얼마가든 루트이가되냐아여기는얼마이 루트이 여기까지 리가따로없으니까그대로에이스얘기는루트이이렇게여러분들께서답을구해주시면되는겁니 1
258
*****************************************************************************

['EBS']
 리꼴이죠그러니까또에이비하고디씨까지등변사 리꼴의성질같 는거까지주어져있습니 그리고길이가삼오칠등이주어져있고요사인비더하기탄젠트비의값그러니까여러분이이런문제를보면이거읽 가이미뭐야되게복잡해보여그런데사인하고탄젠트비의값을어떻게구해한가지원칙만기억하시면돼요뭐예요삼각비구하래요뭐만들어야해직각삼각형이게끝인거야됐나요직각삼각형을만들어야해요여기서이사 리꼴에서그런데각비에대해서니까각비를포함하는직각삼각형을만든 이게그냥핵심이에요그러면각비를포함하는직각삼각형은당연히이렇게만들어야지이거잖아그렇죠그러면여기에서도똑같이그렇죠이거여러분이풀이는워낙유명해왜등변사

['GOOGLE']
 어렵지않은척 음은우리삼번문제로바로이어서들어가보겠습니 레벨보는노편안하게수신부연설좀

514
*****************************************************************************

['EBS']
 그러면누구만연결하면돼요에이비그어볼게요에이비이렇게무슨삼각형직각삼각형바로보이죠그러면사인에이의값을구하라는건지금이거예요맞나요사인은누구분의누구예요에이니까에이비분의비씨입니 정답은누구분의누구야사인에이의값이라는건결국누구를구하라에이비분의비씨를구하라그런데우리는여기에서에이비는조금이거어떻게구하지조금고민되죠당연히고민은되지만누구를쓸거야피타고라스정리쓸거야그렇죠비씨는알아요왜마이너스일에서삼까지잖아비씨의길이는얼마사분자가사예요그렇죠에이비분의사구나나에이비만구해지면답구하는거네에이비는직각삼각형에서의빗변이니까피타고라스정리쓸거야이길이사니까에이씨를구할수있으면돼에이씨도쉽죠왜칠에서부터마이너스일까지의거리니까팔이거든결국두변이사하고팔일때의빗변을구하는겁니

['GOOGLE']
 라고선생님이얘기했던그내용을기억하시면되겠습니  음중옳지않은것같습니 첫번째일번에싸인사십이도사이렌사십이도기준가격있으니까오비분의비에이비에이비에이비가얼마예요일본어결국그대로피에이가될테고비행기가모자피리가영점칠사아니에요여기는여기는이점에와이좌표알면되니까영점육칠이렇게되게일번은옳 나는갈수있는겁니 이번같은경우에싸인사십팔도있어선생님사십팔도없는데요사십팔도이거아닌데누가사십팔곱하기이더하기바로사십팔도아주중가게일로옮겨오는구나그러면싸인사십팔도라는걸이제빗변분의기준가게에서대변인도피니까빗변분의높이오에프에프네오에이이렇게되겠죠옆이사등삼각형에서이편은언제나일이기때문에 른사람중에서빗변분의일일분의영점칠사요일분에영점칠사이렇게바로가주시면끝나는겁니 나오겠죠삼번에이제코사인사십오도있습니 코사인빗변분의이편입니  바로가서일분의누구오에이오에이가얼마야영점칠사맞 이렇게가면되는거예요그 음사번코사인사십오도각도코사인의빗변분의밑변이적일불에요일이뭐야영점칠아니야 시한번요기를영점육칠 됐죠이거이런돈많이쓴 고얘기했어요이렇게해놓고영점칠사라고많이씁니 조심하셔야돼요오번탄젠트는어떤삼각형큰삼각형이것만여러분들이기억하셔서돼요 할수있어요탄젠트사십팔이큰삼각형에서바라보면사십팔돈은

339
*****************************************************************************

['EBS']
 고요됐나요어렵지않게갈수있죠만약에이문제에서코사인엑스를구하세요그러면여기에서뭐하면돼요비디구할수있죠왜피타고라스정리써서루트이에이의제곱더하기에이제곱은이거계산해보면루트오에이가나와요그래서여러분께서뭐코사인엑스면루트오에이분의이에이니까약분하고루트오분의이유리화까지해서오분의이루트오이런식으로답을구하시면되는겁니

['GOOGLE']
 음싸인하고코사인중에서는사십오도일대가값이갖고사십오도보 자를이렇게사십도를대래싸인값이영에서부터출발해서이분의루트이러 가고쿠사이사일에서줄어들고있으니까당연히코사인의값이좀더큰딸안걸려서확인할수있어요그래서니은이들어오고지역이제일작 0
145
*****************************************************************************

['EBS']
 고요됐나요어렵지않게갈수있죠만약에이문제에서코사인엑스를구하세요그러면여기에서뭐하면돼요비디구할수있죠왜피타고라스정리써서루트이에이의제곱더하기에이제곱은이거계산해보면루트오에이가나와요그래서여러분께서뭐코사인엑스면루트오에이분의이에이니까약분하고루트오분의이유리화까지해서오분의이루트오이런식으로답을구하시면되는겁니

['GOOGLE']
 이렇게여러분들께서나일할수있어야되는겁니 됐습니까 음문제로팔번문제들어가보도록하겠습니 이거대소관계를여름도꼭좀그런데또실수많이하는파트입니 좀어려워하고그 음에팔번문제볼게요오늘여름도절대안들립니 그저이십일더하기코사인이십이도에값을구하세요탄젠트어디있어탄젠트이십일도내려와보세요영점삼팔백삼십구영점삼팔삼구더하기새로스퀘어코사인이십이도코사인이십이도어디있습니까영점구육칠이적구천이백칠십이덧셈말실수하지않으면됩니 그래서나일되고요하나올라가서일되고일되고또하나올라가서삼대고일올라가서일점사미터엘쓰는선생님좀빠르게했는데여러분들이사실갔으면더빨리옵니 있으면하지않도록조심하시면되겠습니 레벨온술에문제는지금이렇습니 어떻게자신감이좀생기나요선생님생각이중요한문제라고잡을수있는건당연히육

411
*****************************************************************************

['EBS']
 이이점오도는삼십도사십오도육십도만배웠는데우리가배운적이없죠그런데여러분이이걸이그림을통해서만들어낼수있어야해요무슨이야기냐하면지금이그림을쳐 보잖아요그러면이게이등변삼각형이면이각하고이각이어때같죠맞죠그런데이삼각형에이디비의한외각이사십오도예요한외각은이웃하지않는두내각의합이잖아이렇게둘더해서사십오도라는얘기는사십오도를둘로나눈거니까이거하나가이이점오도씩이란이야기예요그래서탄젠트이이점오도는이제밑변분의높이잖아맞나요그러면똑같은결과를얻을수있단얘기예요그러니까가끔문제에서탄젠트이이점오도의값을구하라는말은이렇게그림이주어지는구나라는걸여러분이생각할수있는데제일어려운문제는그림이당연히없어요그러면여러분은어떤생각을해야하냐면이게지금이등변삼각형직각이등변사십오도를그려놓으면사십오도가외각이되도록하는삼각형을그리는데한외각이되도록그런데나머지두내각의크기가같도록즉이렇게길이가같도록그러면이게이이점오도씩이되겠구나라고그림을생각할수있어야해요그리고여기에서또한가지이각이몇도죠사십오도죠가끔가끔조금더난이도가있

['GOOGLE']
 리꼴이면양쪽있는지삼각형의합동테니까그리고가운데요사각형은수선그러면직사각형일테니까이거유명하잖아그죠요일이얼마삼차칠에서삼을빼면사과남쪽그살을양쪽둘이똑같이나눠가실겁니 살을둘이나눠갖는달리기를요점을에이칠십오났을때요길이가얼마야이 2
421
*****************************************************************************

['EBS']
 면육칠점오도의값을물어볼수도있어요됐나요이각이육칠점오도죠가끔이그림이똑같이주어져놓고탄젠트육칠점오도의값을구하세요이렇게물어보면일분의밑변분의높이잖아이제기준각이이거면그렇죠일분의루트이더하기일그때는답이삼번한문제가지고선생님이너무좀깊이이야기를했는데그만큼이그림이여러분이많이어려워하고

['GOOGLE']
 리꼴잊어그러니까또에이비하고디씨까지등변사 리꼴의성질같 라는것까지주어져있습니 그리고길이가삼백오십칠등이추워져있고요싸인

118
*****************************************************************************

['EBS']
 이렇게여러분께오늘주제꼭오늘것만이아니더라도지금까지배웠던걸차례차례화면별로보여드릴테니까여러분각각의중단원마무리마

['GOOGLE']
 이렇게하면찍어보겠습니 점수이름뭐라고뭐라고되어있나요삼콤마마이너스이엑스좌표가삼백이십아이고이거래상품안해놨어요저이렇게그래야지간단하게해결되는왜지금그러고보니까에이씨는와이축의평이에요그 0
88
*****************************************************************************

['EBS']
 이렇게여러분께오늘주제꼭오늘것만이아니더라도지금까지배웠던걸차례차례화면별로보여드릴테니까여러분각각의중단원마무리마

['GOOGLE']
 음에비쓰는엑스축에평이에요그러기때문에엑스축와이축의수직으로만나니까당연히이에이씨에이치오피스도수직으로만나는그럼누구만연결하면돼비겁격이렇게무슨삼각형직각삼각형바로보이죠그러면싸인값을구하라는건지금요구해요만나요쌓이는누구네누구예요애인이까팔분의빚습니 1
119
*****************************************************************************

['EBS']
 이렇게여러분께오늘주제꼭오늘것만이아니더라도지금까지배웠던걸차례차례화면별로보여드릴테니까여러분각각의중단원마무리마

['GOOGLE']
 아정답은누구누구야에프아이엔에이엘값이라는결국누굴구하라에이비분의비씨를봐라근데우리는여기서조금조금고민되죠당연히고민되지만누구있을것같아걸할수있을거야그죠비씨로알아요왜마이너스일에서삼까지자나얼마나사아그래아분자가사역을하는거네직각삼각형에서의빗변이니가피타고라스정리쓸거야여기서하니까에이스를구할수있으면돼왜붙어마이너스일까지의거리니까파리거든아결국두변이사백구십팔마이너스이빗변을구하는겁니 2
183
****************************************************************************

In [616]:
for e, g in zip(dot_inserted_EBS[:200], dot_inserted_GOOGLE[:200]):
    print(['EBS'])
    print(e)
    print()
    
    print(['GOOGLE'])
    print(g)
    print('*****************************************************************************')
    print()
    

['EBS']
반갑습니다

['GOOGLE']
반갑습니다
*****************************************************************************

['EBS']
  이비에스 친구 여러분들  여러분들과 함께 중학뉴런수학삼하를 강의하게 된 손석민입니다

['GOOGLE']
 이비에스 여러분들 여러분들과 함께 중학뉴런수학 삼할을 강의하기 됨 손성민입니다
*****************************************************************************

['EBS']
 이학년 뉴런을 통해서도 만났었는데요  여러분들  이렇게 다시 만나게 되어서 정말 반갑습니다

['GOOGLE']
 이학년 이런을 통해서도 만났었는데 여러분들 이렇게 다시 만나게 되어서 정말 반갑습니다
*****************************************************************************

['EBS']
 앞으로 삼하 과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거고요 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 삼학년 과정이  하 과정이 특히 어떻게 수업되는지 확인할 수 있을 겁니다

['GOOGLE']
 앞으로 삼과정 어떻게 진행되는지에 대해서는 여러분들께서 오리엔테이션 강의를 통해서 확인할 수 있을 거구요 오리엔테이션 강의는 반드시 듣고 오셔야 여러분들이 삼학년 과정이 하마 종이접기 어떻게 수업 되는지 확인할 수 있을 겁니다
*****************************************************************************

['EBS']
 일단 삼학년 과정 같은 경우에는 크게 삼개의 대단원으로 이루어져 있는데요  첫 번째가 삼각비라는 단원이고 두 번째가 원의 성질  세 번째가 통계  이렇게 이루어져 있습니다

['GOOGLE']
 일단 삼학년 가정 같은 경우에는 크게

In [511]:
ko_pre_EBS[5000:5500] # ~다.에만 점을 찍었음, 전체는 str

'국  이 직각삼각형에서 빗변 분의 밑변의 길이가 얼마니  라는 이 긴 한글을 줄여서 우리는  코사인 에이가 얼마니  이렇게 묻는 거예요  됐나요 세 번째로 밑변 분의 높이의 길이의 비를 우리는 뭐라고 부르냐 하면  탄젠트 라고 부릅니다.  한글로는 됐죠  일단 사인  코사인  탄젠트  그렇죠 탄젠트는 이 삼개 중에 어떤 둘을 뽑아서  밑변 분의 높이를 우리가 계산한 것을 탄젠트 에이라고 이야기를 한다는 얘기예요 그러니까 앞으로  이 직각삼각형에서 에이가 기준각일 때 밑변 분의 높이가 몇 분의 몇이니  이 긴 문장을 줄여서  탄젠트 에이가 얼마니  라고 간단하게 묻는다는 얘기야 수학은 우리가 기호의 약속이죠  그렇죠  새로운 용어가 나오고 새로운 기호가 이렇게 정의되는 구나  라는 걸 알았으면 이제 여러분들이 좀 암기가 필요합니다. 다시 한번 정리하지만 삼각비라는 것은 직각삼각형에서 세 변 중에 두 변을 뽑아서 만들 수 있는 비를 삼각비라고 하고  그 총 육가지 중에 삼가지를 우리가 '

In [513]:
ko_pre_GOOGLE[5000:5500] # ~다.에만 점을 찍었음, 전체는 str

' 알았으면 이제 여러분들이 좀 암기가 필요합니다. 다시 한번 정리 하지만 삼각비라는 것은 세 명 중에 직각삼각형에서 삼번 중에 두 병을 와서 만들 수 있는 길을 상담이라고 하고 그 총 여섯 가지 중에 세 가지를 우리가 중학교 때 됐는데 쌓이는 빗변분의 높이 코사인 은빛  변 탄젠트는 밑변 분의 높이 달하는 이기본 약속을 여러분들이웨어 주신 상태에서 시작을 해야 되는 거예요 그리고 빗변을 집 가게에서 마주보이는 대변이고 높이 뭐라고요 기준가게 대변이 높이다. 그리고 나머지 한 개는 먹어야 된다. 밑변이 된다. 여기까지 여러분들이 먼저 좀 새로운 약속이니까 지금이 약속들을 안 길을 함께 가셔야 됩니다. 나 같이 할게요 내용에 대해서 한번 예제를 좀 살펴 보도록 할 텐데 이렇게 돼 있어요 저 그림과 같이 각도가 구십도인 직각삼각형 에이비씨에서 네모 안에 알맞은 것을 써 넣어라 해놓고 싸긴 에이에스입니다. 차 쌓이는 모였습니까 빗변분의 높이 약속이 오셨다라고 가져갈게요 오늘 수업 끝날 때쯤'

# 전처리끝

# 형태소분석기 시작

#### 아래들만 남기기

- 'NNG' : 보통명사,
- 'NNP': 고유명사,
- 'NNB': 일반의존명사,
- 'NR': 수사,
- 'NP': 대명사, 
- 'VV': 동사,
- 'VA': 형용사,
- 'XR': 어근
- 'SF': 마침표

In [391]:
kkma = Kkma()

In [392]:
ebs_pos = kkma.pos(ko_pre_EBS) #ebs_pos : 리스트

In [393]:
google_pos = kkma.pos(ko_pre_GOOGLE) #google_pos : 리스트

In [394]:
def selected_pos(pos_text):
    alist = []
    for _ in pos_text:
        if _[1] in ['NNG','NNP','NNB','NR','NP', 'VV','VA','XR', 'SF']:
            alist.append(_[0])
            continue
    return alist

In [395]:
list_ebs_pos = selected_pos(ebs_pos)

In [396]:
list_google_pos = selected_pos(google_pos)

In [555]:
integrated_into_W2V = [list_ebs_pos + list_google_pos]

In [574]:
from collections import Counter
c = Counter()

In [579]:
c = Counter(list_ebs_pos)
c.most_common(20) #EBS 형태소 최빈

[('.', 2938),
 ('되', 2666),
 ('하', 1919),
 ('구하', 1805),
 ('있', 1629),
 ('루트', 1485),
 ('에이', 1418),
 ('길이', 1331),
 ('것', 1326),
 ('거', 1200),
 ('삼각형', 1190),
 ('분의', 1151),
 ('뭐', 1136),
 ('사인', 1105),
 ('문제', 1094),
 ('수', 1078),
 ('삼', 1060),
 ('에이치', 1055),
 ('여러분', 1026),
 ('일', 979)]

In [580]:
c = Counter(list_google_pos)
c.most_common(20) #GOOGLE 형태소 최빈

[('.', 2751),
 ('되', 2693),
 ('하', 2470),
 ('있', 1841),
 ('것', 1183),
 ('거', 1156),
 ('삼각형', 1060),
 ('수', 1052),
 ('삼', 1010),
 ('루트', 980),
 ('보', 979),
 ('우리', 934),
 ('일', 922),
 ('에이', 921),
 ('구하', 921),
 ('뭐', 904),
 ('문제', 868),
 ('에이치', 855),
 ('여기', 771),
 ('그러', 757)]

# Skip-gram_W2V

In [595]:
model = Word2Vec(
    integrated_into_W2V, 
    size = 100, #임베딩 차원
    min_count=1, #1번 이상 나온 단어들만
    sg = 1 # skip-gram
)
words = list(model.wv.vocab)
print(len(words))
words[3000:3020]

4348


['교대',
 '중산',
 '노선',
 '커서',
 '종류',
 '스마일',
 '라텍스',
 '특사',
 '떠나',
 '쌍용동',
 '야박',
 '와이지',
 '감기',
 '나비',
 '색도',
 '각설',
 '양면',
 '이론',
 '먹이',
 '살리']

In [534]:
print(model['출력']) #'출력'의 100차원 임베딩 벡터

[-1.5930592e-03 -9.9651504e-04  2.5895892e-03  2.4348267e-03
 -2.2187894e-03 -2.5294907e-03  1.3543374e-04  1.6010102e-03
  2.3477678e-03  2.3956788e-03  1.0304822e-03  3.7424292e-03
  1.8055451e-03 -1.3095014e-03 -3.3443004e-03  2.4683452e-03
 -4.4705905e-04 -2.1984580e-03 -1.8756768e-03  3.8120439e-03
  3.5007421e-03 -4.6892404e-03  2.2645905e-03  4.9117133e-03
 -4.9800077e-03 -7.1900671e-05  1.8555383e-03 -3.1469814e-03
  1.6729099e-03  5.1295053e-04 -3.2042162e-03 -4.5097531e-03
 -4.1223788e-03 -1.7241950e-03 -4.0441286e-04  1.7669275e-03
  3.2149139e-04  2.2709492e-06 -4.9099266e-03  3.0911895e-03
  5.7784910e-04  1.3877428e-03  1.3398062e-03  1.9199456e-03
  4.8026252e-03  2.0271121e-03 -1.8756604e-03  2.7542247e-03
 -3.1167900e-03  4.2773401e-03 -2.5824746e-03  2.2792411e-03
  3.7293914e-03 -1.0675718e-03 -2.8788140e-03  7.1724009e-04
  2.3574519e-03 -6.5433537e-04 -2.9493989e-03 -3.1630713e-03
 -1.5672448e-03 -7.6030503e-04  1.9571276e-03  1.2976634e-03
  4.8236391e-03 -3.26367

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


### 벡터 유사도 확인

In [535]:
model.wv.similar_by_word('콜사인',topn=20)

[('특산', 0.3330569565296173),
 ('송', 0.3248682916164398),
 ('켜지', 0.3102682828903198),
 ('하트', 0.2974466383457184),
 ('기요', 0.29352331161499023),
 ('러브', 0.27738335728645325),
 ('내미', 0.2700957953929901),
 ('늘어지', 0.2692749798297882),
 ('강', 0.26863038539886475),
 ('하나님', 0.26717203855514526),
 ('기부', 0.2669776380062103),
 ('민족', 0.2640741765499115),
 ('입대', 0.25839924812316895),
 ('참가비', 0.2578445076942444),
 ('득', 0.25762632489204407),
 ('하라', 0.2575220465660095),
 ('표기', 0.2548634707927704),
 ('역적', 0.24999502301216125),
 ('계약서', 0.24790675938129425),
 ('인건비', 0.2439521849155426)]

In [536]:
model.wv.similar_by_word('데이트',topn=20) #대입

[('미니', 0.363226056098938),
 ('십이기', 0.33292800188064575),
 ('저장', 0.319174587726593),
 ('라마', 0.31563442945480347),
 ('합격', 0.29744336009025574),
 ('남로', 0.29663240909576416),
 ('내면', 0.29617002606391907),
 ('백은', 0.29093512892723083),
 ('오랜만', 0.276332288980484),
 ('입자', 0.2724442183971405),
 ('도예', 0.27150610089302063),
 ('그려내', 0.27057698369026184),
 ('보관', 0.2630434036254883),
 ('칠하', 0.2621132731437683),
 ('사기', 0.2594667077064514),
 ('파일', 0.2594377100467682),
 ('오전', 0.25632181763648987),
 ('음성', 0.2549046277999878),
 ('여자', 0.2542673945426941),
 ('아이디', 0.250887930393219)]

In [538]:
model.wv.similar_by_word('직각',topn=20)

[('삼각형', 0.9989598393440247),
 ('크', 0.9975802302360535),
 ('작', 0.9971025586128235),
 ('이야기', 0.9949163794517517),
 ('얘기', 0.994428813457489),
 ('구하', 0.9932708740234375),
 ('길이', 0.9932228922843933),
 ('같', 0.9932088851928711),
 ('비가', 0.9925604462623596),
 ('때', 0.9925401210784912),
 ('와이', 0.9924962520599365),
 ('우리', 0.9923476576805115),
 ('경우', 0.9923074245452881),
 ('예', 0.9921680092811584),
 ('자동', 0.9921473860740662),
 ('지금', 0.9920260310173035),
 ('변', 0.9919620156288147),
 ('하래', 0.9917463660240173),
 ('맞', 0.9917344450950623),
 ('도', 0.9915629029273987)]

In [582]:
model.wv.similar_by_word('양면',topn=20) #양변

[('수술', 0.32886919379234314),
 ('상식적', 0.31854870915412903),
 ('양배추', 0.3052181899547577),
 ('아이티', 0.304765522480011),
 ('송편', 0.2985226511955261),
 ('부문', 0.295436829328537),
 ('요정', 0.29494336247444153),
 ('서부', 0.29117634892463684),
 ('잭', 0.28761252760887146),
 ('대충', 0.28524506092071533),
 ('거짓말', 0.27838557958602905),
 ('무난', 0.27823492884635925),
 ('수학책', 0.27145299315452576),
 ('맥박', 0.26976877450942993),
 ('배추', 0.26703140139579773),
 ('꼭지점', 0.2667500078678131),
 ('보조사', 0.26613858342170715),
 ('연회비', 0.2649642825126648),
 ('날씨', 0.26261353492736816),
 ('물건', 0.26178425550460815)]

In [540]:
model.wv.similar_by_word('양배추',topn=20)

[('살리', 0.37294331192970276),
 ('사월', 0.33927881717681885),
 ('양면', 0.3052182197570801),
 ('씻', 0.2954140603542328),
 ('에스디', 0.2929498553276062),
 ('가신', 0.28652703762054443),
 ('기변', 0.2757313847541809),
 ('명', 0.2753024101257324),
 ('뽑아내', 0.2750803828239441),
 ('바위', 0.273049920797348),
 ('시댁', 0.27264606952667236),
 ('노력', 0.2708337903022766),
 ('도요', 0.2694920301437378),
 ('당연', 0.2675219774246216),
 ('재수', 0.2654464840888977),
 ('권하', 0.2652338743209839),
 ('지붕', 0.263318806886673),
 ('이삼십', 0.2608054280281067),
 ('감옥', 0.2601342797279358),
 ('삼명', 0.25598692893981934)]

In [541]:
model.wv.similar_by_word('쓰레기',topn=20) #직'선의기'울기

[('사이클론', 0.3283328711986542),
 ('선보', 0.3194057047367096),
 ('졸리', 0.31625574827194214),
 ('뇌수면', 0.3148439824581146),
 ('스파', 0.29727649688720703),
 ('발휘', 0.2911779284477234),
 ('한', 0.28933194279670715),
 ('가져가', 0.2794867157936096),
 ('입고', 0.2759040296077728),
 ('에스비', 0.2736766040325165),
 ('저녁', 0.2717311382293701),
 ('파면', 0.2669999599456787),
 ('삼과', 0.2644292116165161),
 ('무가', 0.26125043630599976),
 ('쿠', 0.25836724042892456),
 ('꿇', 0.25750210881233215),
 ('계획', 0.2564903497695923),
 ('식이', 0.25424811244010925),
 ('오후', 0.2504929006099701),
 ('잔소리', 0.24822725355625153)]

In [543]:
model.wv.similar_by_word('싸이',topn=20) # sin

[('판매', 0.3305072486400604),
 ('걸이', 0.33035752177238464),
 ('게이', 0.3162769079208374),
 ('팔절', 0.315535306930542),
 ('차비', 0.31528860330581665),
 ('고속도로', 0.2997201681137085),
 ('놓치', 0.28376448154449463),
 ('죄송', 0.28280192613601685),
 ('요부', 0.2812299430370331),
 ('체인', 0.2725892961025238),
 ('분포', 0.27132898569107056),
 ('루틸', 0.2686503529548645),
 ('아양', 0.2672768831253052),
 ('지로', 0.25431588292121887),
 ('깊이', 0.2538769543170929),
 ('각시', 0.2525727450847626),
 ('밝', 0.25096455216407776),
 ('각오', 0.2502482831478119),
 ('슬퍼하', 0.24891844391822815),
 ('구고', 0.24720098078250885)]

In [544]:
model.wv.similar_by_word('임진각',topn=20) #끼인각

[('주제곡', 0.3044273853302002),
 ('엑셀', 0.2987455129623413),
 ('한장석', 0.2931879758834839),
 ('군', 0.2868157625198364),
 ('식당', 0.2842771112918854),
 ('아웃', 0.28154847025871277),
 ('영수', 0.28130829334259033),
 ('아끼', 0.27063700556755066),
 ('산양', 0.2663101553916931),
 ('외부', 0.26275575160980225),
 ('헤어지', 0.2625042796134949),
 ('한', 0.2575348913669586),
 ('타이', 0.2541365921497345),
 ('따라가', 0.24898488819599152),
 ('때리', 0.24664302170276642),
 ('불고', 0.2455018013715744),
 ('하얘지', 0.24486353993415833),
 ('질투', 0.24430693686008453),
 ('세고', 0.24022957682609558),
 ('차비', 0.23857878148555756)]

In [565]:
model.wv.similar_by_word('교대',topn=20) #교재

[('전화', 0.349844366312027),
 ('코스', 0.3383892774581909),
 ('챙기', 0.3335975110530853),
 ('외화', 0.3206803798675537),
 ('특성', 0.3182612657546997),
 ('정수', 0.31174522638320923),
 ('레이블', 0.3098404109477997),
 ('담', 0.3091140687465668),
 ('색도', 0.2909329831600189),
 ('국시', 0.2908720374107361),
 ('중한', 0.28482285141944885),
 ('불의', 0.2770651578903198),
 ('몰골', 0.2764788568019867),
 ('증가량', 0.2759926915168762),
 ('부럽', 0.27521300315856934),
 ('피치', 0.2737242877483368),
 ('식탁', 0.26839810609817505),
 ('하라', 0.2673318386077881),
 ('타오르', 0.2558952271938324),
 ('선물', 0.2551543712615967)]

In [566]:
model.wv.similar_by_word('유기적',topn=20) # 6이죠 [육이죠]

[('옮겨오', 0.39747798442840576),
 ('발음', 0.3747929036617279),
 ('맥주', 0.3232978880405426),
 ('마치', 0.31993332505226135),
 ('그치', 0.2988468110561371),
 ('최신', 0.297924280166626),
 ('맛나', 0.29591497778892517),
 ('오래전', 0.286355197429657),
 ('머리', 0.2787364721298218),
 ('텍스', 0.27412939071655273),
 ('스', 0.2726215720176697),
 ('좀비', 0.26183950901031494),
 ('미지수', 0.26098087430000305),
 ('요구', 0.25859126448631287),
 ('수단', 0.2566666603088379),
 ('야박', 0.2540555000305176),
 ('가세', 0.25186899304389954),
 ('십이기', 0.25108787417411804),
 ('마을', 0.2510238289833069),
 ('터지', 0.24974045157432556)]

In [567]:
model.wv.similar_by_word('사인',topn=20)

[('뭐', 0.9976110458374023),
 ('코', 0.9971542954444885),
 ('탄젠트', 0.9961527585983276),
 ('각', 0.995434582233429),
 ('마주보', 0.9949058294296265),
 ('코사인', 0.9943026304244995),
 ('엑스', 0.9925782084465027),
 ('기준', 0.99152672290802),
 ('주어지', 0.9905422925949097),
 ('변이', 0.990118682384491),
 ('에이', 0.9872025847434998),
 ('비', 0.9853554368019104),
 ('그러', 0.985345721244812),
 ('씨', 0.9842293858528137),
 ('결국', 0.9828837513923645),
 ('빗변', 0.9821886420249939),
 ('누구', 0.9821138978004456),
 ('건', 0.9820030927658081),
 ('오비', 0.9816287159919739),
 ('높이', 0.9815797209739685)]

In [568]:
model.wv.similar_by_word('코사인',topn=20)

[('탄젠트', 0.9988876581192017),
 ('코', 0.9973726272583008),
 ('에이', 0.9955673217773438),
 ('엑스', 0.9954113960266113),
 ('변이', 0.995155394077301),
 ('결국', 0.994742214679718),
 ('그러', 0.9944332838058472),
 ('사인', 0.9943026304244995),
 ('비', 0.9937243461608887),
 ('주어지', 0.9936074614524841),
 ('씨', 0.9935553669929504),
 ('마주보', 0.9932442903518677),
 ('오비', 0.9931861162185669),
 ('삼십', 0.9929121732711792),
 ('누구', 0.9923553466796875),
 ('맞', 0.9923378229141235),
 ('그렇', 0.9923036098480225),
 ('건', 0.9922754168510437),
 ('하면', 0.9918506145477295),
 ('육십', 0.9917073249816895)]

In [569]:
model.wv.similar_by_word('탄젠트',topn=20)

[('코사인', 0.9988876581192017),
 ('코', 0.9986821413040161),
 ('엑스', 0.9975069165229797),
 ('사인', 0.9961527585983276),
 ('변이', 0.9961483478546143),
 ('주어지', 0.9958898425102234),
 ('마주보', 0.9958423376083374),
 ('에이', 0.9947734475135803),
 ('그러', 0.9947336912155151),
 ('비', 0.9941830039024353),
 ('각', 0.9934549927711487),
 ('씨', 0.9933905005455017),
 ('결국', 0.9931511878967285),
 ('뭐', 0.993041455745697),
 ('건', 0.9927048087120056),
 ('누구', 0.992479681968689),
 ('맞', 0.9924598932266235),
 ('오비', 0.9923436045646667),
 ('하면', 0.9915333390235901),
 ('그렇', 0.991435706615448)]

# 아래는 LSTM wikidocs

### https://wikidocs.net/45101 

In [583]:
ebs_before_split = ' '.join(list_ebs_pos).replace(' .', '.')
ebs_morphs_sentence_list = ebs_before_split.split('.')

google_before_split = ' '.join(list_google_pos).replace(' .', '.')
google_morphs_sentence_list = google_before_split.split('.')

intg_mor_sen_list = ebs_morphs_sentence_list + google_morphs_sentence_list

print(len(intg_mor_sen_list))
intg_mor_sen_list[:20]

5691


['반갑',
 ' 이비 에스 친구 여러분 여러분 중학 뉴런 수학 삼하 강의 되 손 석민',
 ' 이 학년 뉴런 통하 만나 여러분 이렇 만나 되 반갑',
 ' 앞 삼하 과정 어떻 진행 지 대하 여러분 오리엔테이션 강의 통하 확인 수 있 거 고요 오리엔테이션 강의 듣 오시 여러분 삼 학년 과정 하 과정 어떻 수업 확인 수 있 것',
 ' 삼 학년 과정 같 경우 크 삼개 대단원 이루어지 번째 삼각비 단원 번째 원 성질 번째 통계 이루어지',
 ' 일 학년 마찬가지 학년 그리하 도형 대하 이야기 삼하 다루 라 걸 생각 되 고요 오늘 시간 삼각비 삼각 알 비 알 합 쳐지 삼각비 새 용어 나오 그러 개념 공부 앞 서 삼각비 뜻 시작 것 시간',
 ' 오늘 시간 시작하 하 터 중학교 삼 학년 수학 손 샘 끝내 공부 내용 가지 개념 오늘 가지 개념 번째 제목 긷 닮 직각 삼각형 대응변 길이 비 것 있',
 ' 그다음 번째 개념 뭐 면 삼각비 것 삼각비 뜻 번째 개념 앞 공부 내용 직각 삼각형 다루 터 직각 삼각형 갖 기본적 성질 있 성질 알 가 자 삼각비 공부 앞서 징검다리 내용',
 ' 여러분 받아들이 되',
 ' 그렇 닮 직각 삼각형 라 하 닮 직각 삼각형 그리 되 닮 직각 삼각형 개 편하 그리 수 있 제일 좋 방법 무엇 까 생각 우리 이렇 그리 되 거 같 한번 그리',
 ' 여기 수선 긋',
 ' 바깥쪽 직각 삼각형 하나 보이',
 ' 그렇 다르 직각 삼각 빗변 위 점 잡 이렇 수선 긋 직 삼각형 에이 비씨 에이디 이 닮음 에이 에이 닮음 되',
 ' 이건 우리 중학교 학년 때 배우 그렇 에이 공통 그다음 둘 직각 갖 에이 에이 닮음 되 터 우리 중이 때 배우 닮음비 것 뭐 하 대응 변 길이 비 그렇 만약 길이 오라 하 보 길이 십 하 삼각형 닮음비 오대 십 대이 그렇 우리 길이 사 노 길이 얼마 되 그렇 사 배 팔 되 되 길이 우리 육 하',
 ' 작 쪽 얼마 되 일대 이대 일 얼마 삼이 되 되 것 여러분 알 수 있 우리 닮음비 것 작 삼각형 크 삼각형 대응 변 길이

In [584]:
text = intg_mor_sen_list
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 4348


In [585]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:20] # 11개의 샘플 출력

[[233, 259],
 [233, 259, 504],
 [233, 259, 504, 20],
 [233, 259, 504, 20, 20],
 [233, 259, 504, 20, 20, 1283],
 [233, 259, 504, 20, 20, 1283, 563],
 [233, 259, 504, 20, 20, 1283, 563, 198],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1, 910],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1, 910, 2074],
 [50, 403],
 [50, 403, 563],
 [50, 403, 563, 217],
 [50, 403, 563, 217, 310],
 [50, 403, 563, 217, 310, 20],
 [50, 403, 563, 217, 310, 20, 31],
 [50, 403, 563, 217, 310, 20, 31, 310],
 [50, 403, 563, 217, 310, 20, 31, 310, 1]]

In [592]:
# t.word_index : 단어 인덱싱
i = 0
for k, v in t.word_index.items():
    print(k, v)
    i+=1
    if i > 30:
        break

되 1
하 2
있 3
구하 4
것 5
루트 6
거 7
에이 8
삼각형 9
수 10
삼 11
뭐 12
길이 13
문제 14
우리 15
에이치 16
일 17
보 18
그러 19
여러분 20
십 21
비 22
분의 23
여기 24
씨 25
사인 26
높이 27
오 28
이것 29
탄젠트 30
이렇 31


In [494]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : 물


In [495]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 436


In [496]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0 ...   0 233 259]
 [  0   0   0 ... 233 259 504]
 [  0   0   0 ... 259 504  20]]


In [497]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [498]:
print(X[:3])

[[  0   0   0 ...   0   0 233]
 [  0   0   0 ...   0 233 259]
 [  0   0   0 ... 233 259 504]]


In [499]:
print(y[:3]) # 레이블

[259 504  20]


In [500]:
vocab_size

4348

In [501]:
y = to_categorical(y, num_classes=vocab_size)

In [502]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [506]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(10))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5, verbose=2)

Epoch 1/5
5187/5187 - 1021s - loss: 5.9883 - accuracy: 0.0324
Epoch 2/5


KeyboardInterrupt: 

In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
sentence_generation(model, t, '이 병 분에 높이', 5)